# Importing necessary libraries

In [61]:
import torch
import cv2
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist
from collections import OrderedDict
import datetime
import numpy as np
import imutils
import csv
import os
from pathlib import Path
%matplotlib inline

# creating a class to track the details of the centriod along with their boxes

In [154]:
class CentroidTracker:
    def __init__(self, maxDisappeared=50, maxDistance=50):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 0
        self.track_time = OrderedDict()
        self.objectclass = OrderedDict()
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.bbox = OrderedDict()  # CHANGE
        self.fileName = ''
        self.final_file_name = ''
        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared

        # store the maximum distance between centroids to associate
        # an object -- if the distance is larger than this maximum
        # distance we'll start to mark the object as "disappeared"
        self.maxDistance = maxDistance

    def register(self, centroid, inputRect,classes):
        # when registering an object we use the next available object
        # ID to store the centroid
        self.objects[self.nextObjectID] = centroid
        self.objectclass[self.nextObjectID] = classes
        self.bbox[self.nextObjectID] = inputRect  # CHANGE
        self.disappeared[self.nextObjectID] = 0
        self.track_time[self.nextObjectID] = datetime.datetime.now()
        self.storeObjectDetails(self.fileName,self.objectclass[self.nextObjectID],self.nextObjectID,self.track_time[self.nextObjectID],' ')
        self.nextObjectID += 1
        

    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        #print(self.objectclass[objectID],self.objects[objectID],self.track_time[objectID],datetime.datetime.now())
        del self.objects[objectID]
        del self.disappeared[objectID]
        del self.bbox[objectID]  # CHANGE
        del self.objectclass[objectID]
        del self.track_time[objectID]
    
    def storeObjectDetails(self,file,name,obectID,startTime,endTime):
        
        if endTime != ' ':
            duration = str(endTime - startTime)
        else: 
            duration =' '
            
        details = ['Object','Id','Display Name','Start Time','End Time','Duration']
        rows = [[name,obectID,'{} {}'.format(name,obectID),startTime,endTime,duration]]
        file_name = '{}_tracked_details.csv'.format(file)
        path = 'Output/{}'.format(file_name)
        self.final_file_name = file_name
        
        if os.path.exists(path):
            with open(path, 'a') as w:
                write = csv.writer(w) 
                write.writerows(rows)
        else:
            with open(path, "w") as f:
                write = csv.writer(f) 
                write.writerow(details) 
                write.writerows(rows)

    def update(self, rects, classes, file_name):
        # check to see if the list of input bounding box rectangles
        # is empty
        self.fileName = file_name
        if len(rects) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.storeObjectDetails(self.fileName,self.objectclass[objectID],objectID,self.track_time[objectID],datetime.datetime.now())
                    self.deregister(objectID)

            # return early as there are no centroids or tracking info
            # to update
            # return self.objects
            return self.bbox, self.objectclass 

        # initialize an array of input centroids for the current frame
        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        inputRects = []
        # loop over the bounding box rectangles
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            # use the bounding box coordinates to derive the centroid
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)
            inputRects.append(rects[i])  # CHANGE

        # if we are currently not tracking any objects take the input
        # centroids and register each of them
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i], inputRects[i],classes[i])  # CHANGE

        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value as at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()

            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]

            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()

            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                if row in usedRows or col in usedCols:
                    continue

                # if the distance between centroids is greater than
                # the maximum distance, do not associate the two
                # centroids to the same object
                if D[row, col] > self.maxDistance:
                    continue

                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.bbox[objectID] = inputRects[col]  # CHANGE
                self.disappeared[objectID] = 0

                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)

            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.storeObjectDetails(self.fileName,self.objectclass[objectID],objectID,self.track_time[objectID],datetime.datetime.now())
                        self.deregister(objectID)

            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col], inputRects[col],classes[col])

        # return the set of trackable objects
        # return self.objects
        return self.bbox, self.objectclass

# creating a function to identify all the boxes when multiple object boxes will be detect at one time

In [16]:
def non_max_suppression_fast(boxes, overlapThresh):
    try:
        if len(boxes) == 0:
            return []

        if boxes.dtype.kind == "i":
            boxes = boxes.astype("float")

        pick = []

        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]

        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(y2)

        while len(idxs) > 0:
            last = len(idxs) - 1
            i = idxs[last]
            pick.append(i)

            xx1 = np.maximum(x1[i], x1[idxs[:last]])
            yy1 = np.maximum(y1[i], y1[idxs[:last]])
            xx2 = np.minimum(x2[i], x2[idxs[:last]])
            yy2 = np.minimum(y2[i], y2[idxs[:last]])

            w = np.maximum(0, xx2 - xx1 + 1)
            h = np.maximum(0, yy2 - yy1 + 1)

            overlap = (w * h) / area[idxs[:last]]

            idxs = np.delete(idxs, np.concatenate(([last],
                                                   np.where(overlap > overlapThresh)[0])))

        return boxes[pick].astype("int")
    except Exception as e:
        print("Exception occurred in non_max_suppression : {}".format(e))


# defining a function to track the video

In [94]:
def track_video(tracker,video,store):
    
    #if user has provided the video path
    if video != 'no':
        
        print('')
        
        #reading video from the given path
        cap = cv2.VideoCapture(video)

        #extracting the video name from the path
        file_name = Path(video).stem
        
        video_name = '{}_tracked_video.avi'.format(file_name)
        
        #if user wants to store tracked video
        if store.lower() == 'y':
            
            #extracting the height and width of the frame of the video
            frame_width = int(cap.get(3))
            frame_height = int(cap.get(4))
            
            #creating object to store the video
            out = cv2.VideoWriter('Output/{}'.format(video_name),cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

        while(True):

            # reading frame from the video
            ret, frame = cap.read()    
            
            if ret == False:
                break
            
            # passing the read frames to the YOLOV5s model
            results = model(frame)

            # finding the x1,y1,x2,y2 coordinates and labels of the bounding boxes of the detected objects in the video from the result object
            cords = results.xyxy[0].numpy()

            boxes = []
            classes = []
            # reading individual box data(box coordinates and classes)
            for c in cords:
                box = c[0:4]
                #print(box)
                boxes.append(box)
                classes.append(results.names[int(c[5])])
            
            boxes = np.array(boxes)
            boxes = boxes.astype("int")
            boxes = non_max_suppression_fast(boxes, 0.3)
            objects, classes = tracker.update(boxes,classes,file_name)

            #reading objectIds and their corresponding boxes and classes 
            for (objectId, bbox) in objects.items():
                
                x1, y1, x2, y2 = bbox
                x1 = int(x1)
                y1 = int(y1)
                x2 = int(x2)
                y2 = int(y2)
                
                #creating boxes arround the objects
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                text = "{} {}".format(classes[objectId],objectId)
                #putting text for the boxes in the video
                cv2.putText(frame, text, (x1, y1-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
                
            if store.lower() == 'y':
                # here writing the video using the out object that we have created above
                out.write(frame)
            cv2.imshow('video',frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        return video_name, tracker.final_file_name
    
    #if user wants for live video
    elif video == 'no':
        
        #reading live video
        cap = cv2.VideoCapture(0)

        #giving name to the video
        file_name = 'live'
        
        video_name = '{}_tracked_video.avi'.format(file_name)
        
        
        #if user wants to store tracked video
        if store.lower() == 'y':
            
            #extracting the height and width of the frame of the video
            frame_width = int(cap.get(3))
            frame_height = int(cap.get(4))
            
            #creating object to store the video
            out = cv2.VideoWriter('Output/{}'.format(video_name),cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

        while(True):

            # reading frame from the video
            ret, frame = cap.read()    
            
            if ret == False:
                break
                
            # passing the read frames to the YOLOV5s model
            results = model(frame)

            # finding the x1,y1,x2,y2 coordinates and labels of the bounding boxes of the detected objects in the video from the result object
            cords = results.xyxy[0].numpy()

            boxes = []
            classes = []
            # reading individual box data(box coordinates and classes)
            for c in cords:
                box = c[0:4]
                #print(box)
                boxes.append(box)
                classes.append(results.names[int(c[5])])
            
            boxes = np.array(boxes)
            boxes = boxes.astype("int")
            boxes = non_max_suppression_fast(boxes, 0.3)
            objects, classes = tracker.update(boxes,classes,file_name)

            #reading objectIds and their corresponding boxes and classes 
            for (objectId, bbox) in objects.items():
                
                x1, y1, x2, y2 = bbox
                x1 = int(x1)
                y1 = int(y1)
                x2 = int(x2)
                y2 = int(y2)
                
                #creating boxes arround the objects
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                text = "{} {}".format(classes[objectId],objectId)
                #putting text for the boxes in the video
                cv2.putText(frame, text, (x1, y1-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
                
            if store.lower() == 'y':
                # here writing the video using the out object that we have created above
                out.write(frame)
            cv2.imshow('Live video',frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        return video_name, tracker.final_file_name
        
        

In [156]:
#creating object of "CentroidTracker" class
tracker = CentroidTracker(maxDisappeared=0, maxDistance=90)

# creating model after loading yolov5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

choice = input('Select one option LIVE VIDEO TRACKING / NORMAL VIDEO TRACKING for LIVE press (L/l) and for NORMAL press (N/n)')
choice = choice.lower()

if os.path.isdir('Output') == False:
    os.mkdir('Output')

if choice == 'n':
    
    video = input('Please enter the video path')
    store = input('Do you want to store tracked video (Y/N)')
    
    if store.lower() == 'n' or store.lower() == 'y':
        video_name,file_name = track_video(tracker,video,store)
        print('Tracked video has been saved with {} name and CSV tracked file has been saved with name {}'.format(video_name,file_name))
    else:
        print('Please choose correct option for store (Y/N)')
        
elif choice == 'l':
    
    video ='no'
    store = input('Do you want to store tracked video (Y/N)')
    
    if store.lower() == 'n' or store.lower() == 'y':
        video_name,file_name = track_video(tracker,video,store)
        print('Tracked video has been saved with {} name and CSV tracked file has been saved with name {}'.format(video_name,file_name))
    else:
        print('Please choose correct option for store (Y/N)')
        
else:
    print('Please select the correct option')
    

Using cache found in C:\Users\MSI 1/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-8-31 torch 1.9.0+cpu CPU

Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 


Select one option LIVE VIDEO TRACKING / NORMAL VIDEO TRACKING for LIVE press (L/l) and for NORMAL press (N/n)l
Do you want to store tracked video (Y/N)y
Tracked video has been saved with live_tracked_video.avi name and CSV tracked file has been saved with name live_tracked_details.csv


In [100]:
dt = datetime.datetime.now()

In [105]:
a = dt.timestamp()-10

In [104]:
print(dt)

2021-09-04 02:53:49.294698


In [108]:
print(datetime.datetime.fromtimestamp(a))

2021-09-04 02:53:39.294698


In [140]:
cap.release()
cv2.destroyAllWindows()